# Lattices and Bases

**Module 08** | 08-lattices-post-quantum

*2D visualization, basis vectors, LLL()*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **lattices and bases**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Basic familiarity with Python syntax.
- A working SageMath installation (or access to CoCalc/SageMathCell).

## What Is a Lattice?

In [ ]:
# A lattice is the set of all integer linear combinations of basis vectors
B = matrix(ZZ, [[1, 0], [0, 1]])
print('Basis:\n', B)

## Visualizing a 2D Lattice

In [ ]:
# Plot lattice points generated by a basis
B = matrix(ZZ, [[3, 1], [1, 2]])
# TODO: plot lattice points as dots in the plane

## Different Bases, Same Lattice

In [ ]:
# Two bases generate the same lattice iff they differ by a unimodular matrix
B1 = matrix(ZZ, [[3, 1], [1, 2]])
U = matrix(ZZ, [[1, 1], [0, 1]])  # unimodular: det = +/- 1
B2 = U * B1
print('B2:\n', B2)
print('det(U):', det(U))

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **lattices and bases**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [The Shortest Vector Problem](08b-shortest-vector-problem.ipynb)